**SI 670 Fall 2020 Kaggle Competition**

Please form teams of up to 4 students.  You should not collaborate with other teams, but you can talk to the GSIs if you are really stuck.  Please record you teams here:  https://docs.google.com/spreadsheets/d/1TDm_bwdSnENU8AMzxjvgqP2Fo3Poi3nRGr8dP1iIbqs/edit?usp=drive_web&ouid=109090332886082450282

For those who filled out the team survey and asked to be assigned to a team, you should find your name and your team members in the Google Sheet above. Please feel free to let us know if we missed you.

Give your team a fun and exciting name for the competition.  You only need one submission for the team.  Everyone on the team will get the same score.  


**Scoring**

There are two tasks, but they will be scored together.  It will be scored out of 100 points.  
You shall receive 40 points for a successful submission.  
You shall receive 20 additional points for attaining benchmark #1.
You shall receive 10 additional points for attaining benchmark #2.
You shall receive 30 * 2 / log2(2 + rank) additional “ranking points”.
The top five teams after the first evaluation will receive an additional 5 points.  

Note that, if you attain both benchmarks, you are guaranteed a score of 82.  The winning team will receive 108 points. 

The competition shall start Tuesday 10/20.

First evaluation is Monday 10/26 at 11:59pm  (only used for 5 bonus points)

Final evaluation is Monday 11/2 at 11:59 pm   

**Context**

Lake monitoring provides important information for environment protection and pollution identification, such as temperature/thermal monitoring. In this Kaggle task, we will provide thermal sensor data for multiple lakes and ask you to predict/estimate the temperature at a certain depth for certain lakes.

Content
We provide Lake Trout’s data during the period of Apr 20th, 2012 - Apr 19th, 2018.

For your reference, we also provide the data of six other lakes (ie., Bear Head, Carlos, Elk, Pearl, Shaokotan, White Iron). Please note that these datasets may vary in terms of the time window (yes, this is how real-world datasets look like!)

For each lake, there will be multiple sensors at different depths, and we will provide lake_id, time, depth and the corresponding temperature at this depth and time point. For example, for the Shaokotan Lake, you will see the following columns, with LakeId representing the lake identifier, Date_time representing the time stamp, Depth_m representing the depth in meters, and Water_Temp_C representing the temperature in Celsius.  Please note that the number of sensors are different across different lakes. 


**Task**

Predicting/estimating the temperature of Trout Lake at depth 10.5 m during Apr 20th, 2018 - Apr 19th, 2019. Please feel free to use part of/all the data we provide. 

Benchmark 1:  MSE < 0.065
Benchmark 2:  MSE < 0.032

Files for first task
For the first task, we will provide you a file Trout_training.csv under the Trout folder which contains all the sensors in the Trout Lake as training. Besides, we will give you another Trout_testing_features.csv under the Trout folder which contains all other sensors’ temperature (except the depth 10.5’s temperature). 

We hope you return a file named Trout_prediction.csv, the dataset you return and upload should have two columns: Date_Time and Water_Temp_C, with a shape of 8,760 * 2. (Reason for 8,760: 8,760 =  365 days * 24 sensorings/day) You are supposed to predict every hour’s temperature of depth-10.5 sensor during  Apr 20th, 2018 - Apr 19th, 2019. 

**Link** 
https://www.kaggle.com/c/si670fall2020/overview


## Task 1

In [45]:
import pandas as pd
import numpy as py

df_trout = pd.read_csv('Greenwood_training.csv')
df_trout.head(10)

,Date_Time,Water_Temp_C,Depth_m
0,2013-12-01 00:00:00,0.687,1.5
1,2013-12-01 00:00:00,0.577,2.5
2,2013-12-01 00:00:00,0.687,3.5
3,2013-12-01 00:00:00,0.687,4.5
4,2013-12-01 00:00:00,0.687,5.5
5,2013-12-01 00:00:00,0.632,6.5
6,2013-12-01 00:00:00,0.715,7.5
7,2013-12-01 00:00:00,0.770,8.5
8,2013-12-01 00:00:00,0.825,9.5
9,2013-12-01 00:00:00,0.742,10.5


In [46]:
df_trout.Date_Time = pd.to_datetime(df_trout.Date_Time)

In [21]:
#df_trout['Water_Temp_C'] = df_trout['Water_Temp_C'].apply(lambda x: np.round(x,6))

In [27]:
# df_trout = df_trout[(df_trout['Date_Time'] >= '2016-04-20') & (df_trout['Date_Time'] <= '2018-04-19')]

In [47]:
month = pd.DatetimeIndex(df_trout.Date_Time).month
df_trout['Month'] = month

# year = pd.DatetimeIndex(df_trout['Date_Time']).year
# df_trout['Year'] = year

# #Extract Hour out of Date_Time
# hour = pd.DatetimeIndex(df_trout['Date_Time']).hour
# df_trout['Hour'] = hour


In [40]:
df_trout.columns

Index(['Date_Time', 'Water_Temp_C', 'Depth_m', 'Month'], dtype='object')

In [44]:
df_trout.index

RangeIndex(start=0, stop=699842, step=1)

In [12]:
lists = [1,2,3,10,11,9]

In [16]:
df_trout = df_trout.dropna()

In [81]:
y=df_trout[df_trout['Month']==12]
x=df_trout[df_trout['Month']!=12]

In [82]:
x = x[['Date_Time', 'Water_Temp_C', 'Depth_m']]

In [83]:
y = y[['Date_Time', 'Water_Temp_C', 'Depth_m']]

In [84]:
# y = unstacked[10.5]
# x = unstacked[lists]
unstacked_x = x.pivot(index='Date_Time', columns='Depth_m', values='Water_Temp_C')
# unstacked_x = unstacked_x.dropna()

In [85]:
unstacked_y = y.pivot(index='Date_Time', columns='Depth_m', values='Water_Temp_C')
# unstacked_y = unstacked_y.dropna()

In [86]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=4)
X_poly = poly.fit_transform(unstacked_x)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [79]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

X_train, X_test, y_poly_train, y_poly_test = train_test_split(X_poly, unstacked_y,  random_state= 2666)
scaler = MinMaxScaler()
standardized_X_poly_train = scaler.fit_transform(X_train)
standardized_X_poly_test = scaler.transform(X_test) 


ValueError: Found input variables with inconsistent numbers of samples: [31717, 2964]

In [320]:
from sklearn.linear_model import Lasso, Ridge, LinearRegression
import numpy as np
from sklearn.metrics import mean_squared_error
alpha_list = [0.0000001,0.0000001, 0.001, 0.005, 0.01, 0.1]
mselist = []

for alpha in alpha_list:
    linlasso = Ridge(alpha, max_iter = 20000).fit(standardized_X_poly_train, y_poly_train)
    y_pred = linlasso.predict(standardized_X_poly_test)
    mse = mean_squared_error(y_poly_test, y_pred)
    mselist.append(mse)
    

In [321]:
mselist

[0.02100767569715769,
 0.02100767569715769,
 0.027472651047290447,
 0.028532448157811145,
 0.029059752409476017,
 0.030546507016760933]

In [322]:
linlasso = Ridge(0.0000001, max_iter = 10000).fit(standardized_X_poly_train, y_poly_train)
y_pred = linlasso.predict(standardized_X_poly_test)

In [323]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_poly_test, y_pred)

0.02100767569715769

In [324]:
ans = pd.read_csv("Trout_testing_features.csv")

In [325]:
predict = ans[['Date_Time', "Water_Temp_C", 'Depth_m']]

In [326]:
predict = predict.pivot(index='Date_Time', columns='Depth_m', values='Water_Temp_C')
predict= predict.dropna()

In [327]:
predict = predict[lists]

In [328]:
predict

Depth_m,6.5,7.5,8.5,9.5,11.5,12.5,14.5
Date_Time,,,,,,,
2018-04-20 00:00:00,0.989,1.289,1.289,1.453,1.561,1.534,1.724
2018-04-20 01:00:00,0.989,1.289,1.289,1.453,1.561,1.534,1.724
2018-04-20 02:00:00,0.989,1.289,1.289,1.453,1.561,1.534,1.724
2018-04-20 03:00:00,0.989,1.289,1.289,1.453,1.561,1.534,1.724
2018-04-20 04:00:00,0.989,1.289,1.289,1.453,1.561,1.534,1.724
...,...,...,...,...,...,...,...
2019-04-19 19:00:00,3.485,3.617,3.512,3.617,3.591,3.564,3.696
2019-04-19 20:00:00,3.459,3.617,3.512,3.617,3.617,3.564,3.696
2019-04-19 21:00:00,3.485,3.617,3.512,3.617,3.591,3.564,3.696


In [329]:
# poly = PolynomialFeatures(degree=3)
X_pre = poly.fit_transform(predict)

In [330]:
X_pre

array([[  1.        ,   0.989     ,   1.289     , ...,   6.99399379,
          7.8602642 ,   8.83383017],
       [  1.        ,   0.989     ,   1.289     , ...,   6.99399379,
          7.8602642 ,   8.83383017],
       [  1.        ,   0.989     ,   1.289     , ...,   6.99399379,
          7.8602642 ,   8.83383017],
       ...,
       [  1.        ,   3.485     ,   3.617     , ..., 173.51591543,
        179.94243082, 186.60696529],
       [  1.        ,   3.485     ,   3.617     , ..., 173.51591543,
        179.94243082, 186.60696529],
       [  1.        ,   3.485     ,   3.617     , ..., 170.9934923 ,
        178.62971948, 186.60696529]])

In [331]:
standardized_X_poly_pre = scaler.transform(X_pre) 

In [332]:
ans_pred = linlasso.predict(standardized_X_poly_pre)

In [333]:
pred = pd.DataFrame(ans_pred)

In [334]:
pred

,0
0,1.388327
1,1.388327
2,1.388327
3,1.388327
4,1.388327
...,...
8755,3.586961
8756,3.599167
8757,3.586961
8758,3.586961


In [335]:
file_name = 'Trout_prediction2.csv'

In [336]:
pred.to_csv(file_name)